## Data Read-in

In [82]:
!pip uninstall kaleido -y
!pip install --upgrade "kaleido==0.1.*"
import kaleido #required
print(kaleido.__version__) #0.2.1

import os
import glob
import regex as re
import json
from urllib.request import urlopen
import requests
import time
from geopy import Nominatim
import sqlite3
# dash
import dash
from dash import Dash, dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State

# plotting and df
import plotly
print(plotly.__version__)
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
import pandas as pd 

import io
import PIL
r = np.random.RandomState(42)


Found existing installation: kaleido 0.1.0
Uninstalling kaleido-0.1.0:
  Successfully uninstalled kaleido-0.1.0
  Using cached kaleido-0.1.0-py2.py3-none-macosx_10_10_x86_64.whl (69.5 MB)
0.2.1
5.13.0


In [322]:
from matplotlib import pyplot as plt

1. Time-series data of fundings in different research areas (dollars in millions and rounded)
from 2017 - 2023
https://report.nih.gov/funding/categorical-spending#/ 



In [2]:
fund_sum = pd.read_excel("RCDCFundingSummary_02172023.xlsx")

In [3]:
fund_sum.head()

,"Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC)","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).1","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).2","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).3","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).4","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).5","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).6","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).7","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).8","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).9",...,"Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).11","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).12","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).13","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).14","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).15","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).16","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).17","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).18","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).19","Estimates of Funding for Various Research, Condition, and Disease Categories (RCDC).20"
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Research/Disease Areas \n (Dollars in millions...,2008,2009,2009 ARRA,2010,2010 ARRA,2011,2012,2013,2014,...,2016,2017,2018,2019,2020,2021,2022 Estimated,2023 Estimated,2019 US Mortality 19,2019 US Prevalence SE 19
2,Acquired Cognitive Impairment,+,+,+,+,+,+,+,+,+,...,1132,1560,1978,2423,2897,3259,3555,3354,-,-
3,Acute Respiratory Distress Syndrome,82,103,17,110,22,96,98,95,85,...,103,107,123,126,158,148,159,141,10482,-
4,Adolescent Sexual Activity,+,+,+,80,7,69,76,70,68,...,91,99,96,102,88,94,92,94,-,-


2. NIH Awards: by project names, prpoject number (Activity code included) Location, Organization, admin IC, funding mechanisms, direct costs, and indirect costs per fiscal year and project number 
https://report.nih.gov/award/index.cfm?ot=&fy=2023&state=&ic=&fm=&orgid=&distr=&rfa=&om=n&pid=#tab5 
\

We import a city FIPS table for geospatial data display

In [309]:
cfips = pd.read_csv("county-fips.csv")
cfips["full_name"] = (cfips["county_name"]+" "+cfips["state_name"]).str.upper()
fip_dict = dict(zip(list(cfips["full_name"]),list(cfips["fips"])))
list(fip_dict.items())[1:5]


[('BALDWIN COUNTY ALABAMA', 1003),
 ('BARBOUR COUNTY ALABAMA', 1005),
 ('BIBB COUNTY ALABAMA', 1007),
 ('BLOUNT COUNTY ALABAMA', 1009)]

The following would be a dict to map state full name to state two-letter code. 

In [310]:
anym = pd.read_csv("acronynm.csv")
c_dict = dict(zip(list(anym.state.str.upper()),list(anym.code)))

In [362]:

data = pd.DataFrame()
path = "/Users/linlin/Desktop/2023/16b/dash/awardsbyloc"
xls_list = os.listdir(path)
xlsx = glob.glob(os.path.join(path, "*.xls"))
for x in xlsx:
    if '.DS_Store' not in x:
        print(x)
        df = pd.read_excel(x)
        df.columns = [s.upper() for s in list(df.columns)]
        df["MONTH"] = pd.to_datetime(df["AWARD NOTICE DATE"]).dt.month
        df["YEAR"] = pd.to_datetime(df["AWARD NOTICE DATE"]).dt.year
        df["CITY"] = df["CITY"].str.upper()
        df["STATE OR COUNTRY NAME"] = df["STATE OR COUNTRY NAME"].str.upper()
        df["FULL_LOC"] = df["CITY"] + " " + df["STATE OR COUNTRY NAME"]
        df['CODE'] = df["STATE OR COUNTRY NAME"].map(c_dict)
        
        if len(data) == 0:
            data = df
        else:
            data = pd.concat([data, df],    # Combine vertically
                            ignore_index = False,
                            sort = False)

        

/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2018.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2019.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2021.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2020.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2022.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2012.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2013.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2014.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2015.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2017.xls
/Users/linlin/Desktop/2023/16b/dash/awardsbyloc/Worldwide2016.xls


In [371]:
data.head()

,ORGANIZATION NAME,ORGANIZATION ID (IPF),PROJECT NUMBER,FUNDING MECHANISM,NIH REFERENCE,PI NAME,PI PERSON ID,PROJECT TITLE,DEPT NAME,NIH DEPT COMBINING NAME,...,MEDICAL SCHOOL LOCATION,INSTITUTION TYPE,AWARD NOTICE DATE,FOA,MONTH,YEAR,FULL_LOC,CODE,NIH DEPT COMBINNING NAME,MEDICAL SCHOOL
0,"21ST CENTURY MEDICINE, INC.",4238201.0,1R43HD097690-01,RPGs - SBIR/STTR,9672321,"TING, ALISON Y",10504815.0,Whole Ovary Vitrification for Fertility Preser...,NaN,NONE,...,NaN,None,2018-09-25,PA18-574,9.0,2018.0,FONTANA CALIFORNIA,CA,NaN,NaN
1,"21ST CENTURY MEDICINE, INC.",4238201.0,1R43DK117782-01,RPGs - SBIR/STTR,9558420,"FAHY, GREGORY M",6580461.0,Development of Apparatus for the Vitrification...,NaN,NONE,...,NaN,None,2018-09-17,PA17-302,9.0,2018.0,FONTANA CALIFORNIA,CA,NaN,NaN
2,"21ST CENTURY THERAPEUTICS, INC.",4524201.0,1R43HL139196-01A1,RPGs - SBIR/STTR,9541093,"SHAW, JIAJIU",1980890.0,Development of a Novel Small Molecule as a Rad...,NaN,NONE,...,NaN,None,2018-09-16,PA17-302,9.0,2018.0,DETROIT MICHIGAN,MI,NaN,NaN
3,3-C INSTITUTE FOR SOCIAL DEVELOPMENT,4326801.0,1R43HD094420-01A1,RPGs - SBIR/STTR,9556059,"DEROSIER, MELISSA E",8194555.0,Serious game for bullying prevention to prepar...,NaN,NONE,...,NaN,Research Institutes,2018-08-03,PA17-302,8.0,2018.0,DURHAM NORTH CAROLINA,NC,NaN,NaN
4,3-C INSTITUTE FOR SOCIAL DEVELOPMENT,4326801.0,4R44HD090805-02,RPGs - SBIR/STTR,9644106,"DEROSIER, MELISSA E",8194555.0,Development and evaluation of a serious game f...,NaN,NONE,...,NaN,Research Institutes,2018-08-24,PA16-302,8.0,2018.0,DURHAM NORTH CAROLINA,NC,NaN,NaN


In [416]:
data['INSTITUTION TYPE']= data['INSTITUTION TYPE'].fillna('None')

In [417]:
# store data in database data.db table name "decade"

conn = sqlite3.connect("data.db")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS decade")
conn.commit()
data.to_sql("decade",conn,index=False)

623825

In [418]:

print(pd.read_sql_query("SELECT * FROM decade LIMIT 1",conn))
conn.close()

             ORGANIZATION NAME  ORGANIZATION ID (IPF)   PROJECT NUMBER  \
0  21ST CENTURY MEDICINE, INC.              4238201.0  1R43HD097690-01   

  FUNDING MECHANISM  NIH REFERENCE         PI NAME  PI PERSON ID  \
0  RPGs - SBIR/STTR        9672321  TING, ALISON Y    10504815.0   

                                       PROJECT TITLE DEPT NAME  \
0  Whole Ovary Vitrification for Fertility Preser...      None   

  NIH DEPT COMBINING NAME  ... MEDICAL SCHOOL LOCATION INSTITUTION TYPE  \
0                    NONE  ...                    None             None   

     AWARD NOTICE DATE       FOA MONTH    YEAR            FULL_LOC CODE  \
0  2018-09-25 00:00:00  PA18-574   9.0  2018.0  FONTANA CALIFORNIA   CA   

  NIH DEPT COMBINNING NAME MEDICAL SCHOOL  
0                     None           None  

[1 rows x 29 columns]


In [49]:
# retrieving location: longitude and latitude based on city and state

geolocator = Nominatim(user_agent="app")

#county code
https://raw.githubusercontent.com/plotly/dash-sample-apps/main/apps/dash-opioid-epidemic/data/lat_lon_counties.csv


In [372]:
# just downloaded the county and city fips code from us census data and mapped
city = pd.read_csv("uscities.csv")
city.city = city.city.str.upper()
city.county_name = city.county_name.str.upper()
city.state_name = city.state_name.str.upper()
city = city.dropna()
# store data in database data.db table name "decade"
conn = sqlite3.connect("data.db")
city.to_sql("city",conn,index=False)
conn.close()


In [373]:
data.columns

Index(['ORGANIZATION NAME', 'ORGANIZATION ID (IPF)', 'PROJECT NUMBER',
       'FUNDING MECHANISM', 'NIH REFERENCE', 'PI NAME', 'PI PERSON ID',
       'PROJECT TITLE', 'DEPT NAME', 'NIH DEPT COMBINING NAME',
       'NIH MC COMBINING NAME', 'DIRECT COST', 'INDIRECT COST', 'FUNDING',
       'CONGRESSIONAL DISTRICT', 'CITY', 'STATE OR COUNTRY NAME', 'ZIP CODE',
       'ATTRIBUTED TO MEDICAL SCHOOL', 'MEDICAL SCHOOL LOCATION',
       'INSTITUTION TYPE', 'AWARD NOTICE DATE', 'FOA', 'MONTH', 'YEAR',
       'FULL_LOC', 'CODE', 'NIH DEPT COMBINNING NAME', 'MEDICAL SCHOOL'],
      dtype='object')

In [375]:
data['FUNDING MECHANISM'].unique()

array(['RPGs - SBIR/STTR', 'RPGs - Non SBIR/STTR', 'R&D Contracts',
       'Other Research-Related', 'Training - Individual',
       'Training - Institutional', 'Research Centers', 'Other',
       'Construction'], dtype=object)

In [415]:
data['INSTITUTION TYPE'].unique()

array(['None', 'Research Institutes', 'Domestic Higher Education',
       'Independent Hospitals'], dtype=object)

In [374]:
city.columns

Index(['city', 'city_ascii', 'state_id', 'state_name', 'county_fips',
       'county_name', 'lat', 'lng', 'population', 'density', 'source',
       'military', 'incorporated', 'timezone', 'ranking', 'zips', 'id'],
      dtype='object')

In [387]:
cmd = '''
SELECT D.[PROJECT NUMBER], D.[FUNDING MECHANISM],D.[DIRECT COST],D.[INDIRECT COST],
D.FUNDING,D.CITY,D.[STATE OR COUNTRY NAME],D.[INSTITUTION TYPE],D.YEAR,D.MONTH,
D.FULL_LOC,D.CODE,COUNTY_FIPS,C.COUNTY_NAME,C.LAT,C.LNG
FROM 
        (SELECT LNG,LAT,COUNTY_NAME,CITY,COUNTY_FIPS,STATE_ID
        FROM city) AS C
LEFT JOIN decade as D
ON D.CITY = C.CITY
WHERE year =:year AND month=:month AND D.[FUNDING MECHANISM]=:fme
        AND D.[INSTITUTION TYPE]=:inst
LIMIT 10
'''
#land size filter
year = 2022
fme = "R&D Contracts"
inst = 'Domestic Higher Education'
month = 11
conn = sqlite3.connect("data.db")
cursor = conn.cursor()
cursor.execute(cmd,{'year': year,  'month': month, 'inst': inst, 'fme': fme
})
df = pd.DataFrame(cursor.fetchall())
print(df.head())
conn.close()

Empty DataFrame
Columns: []
Index: []


In [426]:
def map_que(year=list(np.arange(2012,2022)),month=list(np.arange(1,12)),
            fme = list(data['INSTITUTION TYPE'].unique()),
            inst = list(data['FUNDING MECHANISM'].unique())):
        cmd = '''
        SELECT D.[PROJECT NUMBER], D.[FUNDING MECHANISM],D.[DIRECT COST],D.[INDIRECT COST],
        D.FUNDING,D.CITY,D.[STATE OR COUNTRY NAME],D.[INSTITUTION TYPE],D.YEAR,D.MONTH,
        D.FULL_LOC,D.CODE,COUNTY_FIPS,C.COUNTY_NAME,C.LAT,C.LNG
        FROM 
                (SELECT LNG,LAT,COUNTY_NAME,CITY,COUNTY_FIPS,STATE_ID
                FROM city) AS C
        LEFT JOIN decade as D
        ON D.CITY = C.CITY
        WHERE D.YEAR IN ''' + str(tuple(year))\
        + ''' AND MONTH IN ''' + str(tuple(month))\
        + ''' AND D.[FUNDING MECHANISM] IN ''' + str(tuple(inst))\
        + ''' AND D.[INSTITUTION TYPE] IN ''' + str(tuple(fme))
        #print(cmd)
        conn = sqlite3.connect("data.db")
        cursor = conn.cursor()

        cursor.execute(cmd)
        df = pd.DataFrame(cursor.fetchall(),columns=["PROJECT NUMBER", "FUNDING MECHANISM","DIRECT COST","INDIRECT COST",
        "FUNDING","CITY","STATE OR COUNTRY NAME","INSTITUTION TYPE","YEAR","MONTH",
        "FULL_LOC","CODE","COUNTY_FIPS","COUNTY_NAME","LAT","LNG"])
        print(df.head())
        conn.close()
        return df


In [427]:
df = map_que()

      PROJECT NUMBER       FUNDING MECHANISM  DIRECT COST  INDIRECT COST  \
0    1C06OD018416-01            Construction   29376655.0            NaN   
1    1C06OD018417-01            Construction   31459563.0            NaN   
2    1C06OD030152-01            Construction    8000000.0            NaN   
3  1D43TW009588-01A1  Other Research-Related     282000.0        22400.0   
4    1D43TW009604-01  Other Research-Related     259200.0        16760.0   

    FUNDING      CITY STATE OR COUNTRY NAME           INSTITUTION TYPE  \
0  29376655  NEW YORK              NEW YORK  Domestic Higher Education   
1  31459563  NEW YORK              NEW YORK  Domestic Higher Education   
2   8000000  NEW YORK              NEW YORK  Domestic Higher Education   
3    304400  NEW YORK              NEW YORK  Domestic Higher Education   
4    275960  NEW YORK              NEW YORK                       None   

     YEAR  MONTH           FULL_LOC CODE  COUNTY_FIPS COUNTY_NAME      LAT  \
0  2014.0    4.0  NE

In [434]:
def geo_encode(dat,fip_dict):
    proj = dat.copy()
    #BECAUSE THERE ARE DUPLICATED VALUES AND USUALLY THE FIRST OCCURENCES ARE RIGHT
    locs = list(dat.FULL_LOC)
    lngs =  list(dat.LNG)
    lats = list(dat.LAT)
    fips = list(dat.COUNTY_FIPS)
    countys = list(dat.COUNTY_NAME)
    
    lon_dict = dict(zip(list(locs),list(lngs)))
    lat_dict = dict(zip(list(locs),list(lats)))
    ct_dict = dict(zip(list(locs),list(countys)))
    fip_dict = dict(zip(list(locs),list(fips)))
    notfound = (proj[proj["LAT"].isna()]["FULL_LOC"]).unique()
    #print(notfound)

    geo = [geolocator.geocode(i) for i in notfound] 
    lat = [i.latitude if i is not None else np.nan for i in geo]
    lon = [i.longitude if i is not None else np.nan for i in geo]
    county = [i.address.upper().split(", ")[1] if i is not None else np.nan for i in geo]
    fip = [fip_dict.get(i.address.upper().split(", ")[1] + " " +i.address.upper().split(", ")[2]) \
           if i is not None else np.nan 
           for i in geo]
    
    notfoundl = list(notfound)
    lat_add = dict(zip(notfoundl,lat))
    lon_add = dict(zip(notfoundl,lon))
    ct_add = dict(zip(notfoundl,county))
    fip_add = dict(zip(notfoundl,fip))

    ct_dict.update(ct_add)
    fip_dict.update(fip_add)
    lat_dict.update(lat_add)
    lon_dict.update(lon_add)
    
    proj["COUNTY"] = proj["FULL_LOC"].map(ct_dict)
    proj["FIPS"] = proj["FULL_LOC"].map(fip_dict)
    proj["LNG"] = proj["FULL_LOC"].map(lon_dict)
    proj["LAT"] = proj["FULL_LOC"].map(lat_dict)
    return proj 



In [435]:

proj = geo_encode(df,fip_dict)

In [438]:
# store data in database data.db table name "decade"

conn = sqlite3.connect("data.db")
cur = conn.cursor()
cur.execute("DROP TABLE IF EXISTS decade")
conn.commit()
proj.to_sql("decade",conn,index=False)
conn.close()


In [439]:
proj.to_csv("abl_convert.csv")

In [13]:
proj = pd.read_csv("abl_convert.csv")

In [36]:
proj.columns

Index(['Unnamed: 0', 'PROJECT NUMBER', 'FUNDING MECHANISM', 'DIRECT COST',
       'INDIRECT COST', 'FUNDING', 'CITY', 'STATE OR COUNTRY NAME',
       'INSTITUTION TYPE', 'YEAR', 'MONTH', 'FULL_LOC', 'CODE', 'COUNTY_FIPS',
       'COUNTY_NAME', 'LAT', 'LNG', 'COUNTY', 'FIPS'],
      dtype='object')

In [96]:
fs = proj.groupby(['CODE','YEAR','FUNDING MECHANISM'])['FUNDING'].sum()

In [97]:
fs = fs.reset_index(0).reset_index(0).reset_index(0)

In [23]:
fsdf = pd.DataFrame({"CODE":fs.index,"FUNDING":fs.values})
fsdf["FUND.NORM"] = (fsdf["FUNDING"] - fsdf["FUNDING"].min()) / (fsdf["FUNDING"].max() - fsdf["FUNDING"].min())
fsdf["CODE"].unique()

array(['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI',
       'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV',
       'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VA', 'VT', 'WA', 'WI', 'WV', 'WY'], dtype=object)

In [87]:
cs = proj.groupby(['COUNTY_FIPS','YEAR','FUNDING MECHANISM'])['FUNDING'].sum()

In [88]:
cs = cs.reset_index(0).reset_index(0).reset_index(0)

In [32]:
#csdf = pd.DataFrame({"FIPS":cs.index,"FUNDING":cs.values})

In [ ]:

with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)
fig = px.choropleth_mapbox(csdf, geojson=counties, locations='FIPS', color='FUNDING',
                           color_continuous_scale="Viridis",
                           #color_continuous_scale=px.colors.diverging.BrBG,
                           color_continuous_midpoint=np.mean(csdf.FUNDING),
                           #range_color=(0, 0.5),
                           mapbox_style="carto-positron",
                           zoom=3, center = {"lat": 36.5, "lon": -95},
                           opacity=0.5,
                           labels={'FUND.NORM':'min-max norm. TOTAL PROJECT FUNDING'}
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [72]:
kaleido.__version__

'0.2.1'

In [99]:
fig = px.choropleth(fs,locations="CODE", locationmode="USA-states", color="FUNDING", scope="usa",animation_group="YEAR", animation_frame="FUNDING MECHANISM",range_color=(0, 1.5*10e9),width=800,height=500)
fig.show()



generate images for each step in animation
frames = []
for s, fr in enumerate(fig.frames):
    # set main traces to appropriate traces within plotly frame
    fig.update(data=fr.data)
    # move slider to correct place
    fig.layout.sliders[0].update(active=s)
    # generate image of current state
    frames.append(PIL.Image.open(io.BytesIO(fig.to_image(format="png",engine='kaleido'))))
    
create animated GIF
frames[0].save(
        "test.gif",
        save_all=True,
        append_images=frames[1:],
        optimize=True,
        duration=500,
        loop=0,
    )

In [166]:
rcdc = pd.read_excel("RCDCFundingSummary_03072023.xlsx",)
rcdc.columns = rcdc.loc[1,:]
rcdc = rcdc.loc[2:310,:]


In [308]:
selected = rcdc[rcdc['2019 US Mortality 19']!='-']
selected = selected.applymap(lambda x:str(x).replace('+','-1'))
selected = selected.applymap(lambda x:str(x).replace('$',''))

selected[2009]=selected[2009].astype(float) + selected['2009 ARRA'].astype(float)
selected[2010]=selected[2010].astype(float) + selected['2010 ARRA'].astype(float)
selected.drop(['2009 ARRA','2010 ARRA'],axis=1,inplace=True)


In [309]:
selected['exist'] = selected.apply((lambda row: sum([1 if (row[x] != '-1') else 0 for x in list(selected.columns)[1:-3]])),axis = 1)
selected['avg'] = selected.apply((lambda row: np.mean([int(row[x]) if (row[x] != '-1') else 0 for x in list(selected.columns)[1:-3]])),axis = 1)
selected['estimate'] = selected['2023 Estimated'].astype(float) - selected['2022 Estimated'].astype(float) 
selected['2019 US Mortality 19'] = selected['2019 US Mortality 19'].astype('float')
selected["pct2023"] = selected.loc[:,['2022 Estimated','2023 Estimated']].astype(float).pct_change(axis=1)['2023 Estimated'].multiply(100)
selected =selected.sort_values('2019 US Mortality 19')
selected.head()
#rate of change with linear regression

1,Research/Disease Areas \n (Dollars in millions and rounded),2008,2009,2010,2011,2012,2013,2014,2015,2016,...,2020,2021,2022 Estimated,2023 Estimated,2019 US Mortality 19,2019 US Prevalence SE 19,exist,avg,estimate,pct2023
277,Temporomandibular Muscle/Joint Disorder (TMJD),19,16.0,17.0,18,21,19,18,16,15,...,17,14,14,14,1.0,-,15,16.3750,0.0,0.000000
158,Lead Poisoning,9,14.0,12.0,10,8,8,7,7,9,...,11,9,9,10,2.0,-,15,9.6250,1.0,11.111111
244,Scleroderma,20,23.0,21.0,25,24,21,24,22,18,...,24,21,22,22,8.0,-,15,21.9375,0.0,0.000000
172,Malaria,132,121.0,148.0,145,152,147,169,163,183,...,211,229,239,236,9.0,-,15,179.0625,-3.0,-1.255230
23,Ataxia Telangiectasia,13,15.0,13.0,13,11,11,8,7,7,...,10,10,10,10,14.0,-,15,10.3750,0.0,0.000000


In [293]:
px.histogram(selected['2019 US Mortality 19'],nbins=500)

In [338]:

mort_bins = pd.IntervalIndex.from_tuples([(0, 50*10e2),(50*10e2, 99*10e2), (99*10e2, 100*10e2),
                                        (100*10e2, 150*10e2),
                                        (150*10e2, 200*10e2),
                                        (200*10e2, 250*10e2),
                                        (250*10e2, 300*10e2),
                                        (300*10e2, 350*10e2),
                                        (350*10e2, 400*10e2),
                                        (400*10e2, 1*10e6),
                                        (10e6, 2*10e6)])
mort_dict = dict(zip([str(i) for i in list(mort_bins)],['<5k','5-10k','10-100k',
                                 '100-150k',
                                 '150-200k',
                                 '200-250k',
                                 '250-300k',
                                 '300-350k',
                                 '350-400k',
                                 '400k - 1M',
                                 '> 1M']))

In [343]:
px.histogram(selected['pct2023'])

In [350]:
pct_bins = pd.IntervalIndex.from_tuples([(-300,-20),(-20, -10),(-10,-5), (-5,0),
                                        (0,5),
                                        (5,10),
                                        (10,15),
                                        (15,20),
                                        (20,300)])
pct_dict = dict(zip([str(i) for i in list(pct_bins)],['< -20','-20 ~ -10','-10 ~ -5', '-5 ~ 0',
                                                       '0 ~ 5',
                                                       '5 ~ 10',
                                                       '10 ~ 15',
                                                       '15 ~ 20',
                                                       '> 20']))
selected['est_bin'] = pd.cut(selected['estimate'], pct_bins).astype(str)
selected['mort_bin'] = pd.cut(selected['2019 US Mortality 19'], mort_bins).astype(str)
selected['mort_str'] = selected['mort_bin'].astype(str).map(mort_dict)
selected['est_str'] = selected['est_bin'].astype(str).map(pct_dict)

In [381]:
select = selected.sort_values('2019 US Mortality 19').iloc[-10:,:]

In [352]:
#df = px.data.tips()
fig = px.parallel_categories(selected, dimensions=['exist','est_str','mort_str'],
                color="exist", color_continuous_scale="Viridis",
                labels={'exist':'Category existed time (yr)',
                        'est_str':'Estimated pct. change',
                        'mort_str':'Mortality rate (raw ct.)'},
                height = 500,
                width = 900
                )
dimensions=['exist','est_str','mort_bin']
fig.update_traces(dimensions=[{"categoryorder": "category descending"} for _ in dimensions])
plt.tight_layout()
fig.show()

<Figure size 432x288 with 0 Axes>

In [382]:
sem = pd.melt(select, id_vars=['Research/Disease Areas \n (Dollars in millions and rounded)'], 
        value_vars=selected.columns[1:17],
        var_name='year', value_name='value')
sem = sem.rename(columns={'2022 Estimated': '2022','2023 Estimated': '2023'})
sem['value'] = sem['value'].astype(int)

In [386]:
fig = px.bar(sem, x='Research/Disease Areas \n (Dollars in millions and rounded)', 
             y="value", color='Research/Disease Areas \n (Dollars in millions and rounded)',
  animation_frame="year",width = 1000,height = 600,range_y=[0,8000])
fig.show()

3. Activity code: categories of fundigns with titles, descriptions, and activity code 
NIH uses activity codes (e.g. R01, R43, etc.) to differentiate the wide variety of research-related programs we support. NIH Institutes and Centers (ICs) Link to External Site may vary in the way they use activity codes; not all ICs accept applications for all types of grant programs or they apply specialized eligibility criteria. Look closely at Funding Opportunity Announcements (FOAs) to determine which ICs participate and the specifics of eligibility.
https://grants.nih.gov/grants/funding/ac_search_results.htm



In [19]:
codes = pd.read_excel("ActivityCodes.xls")

In [20]:
codes.head()

,ID,ACT_CATEGORY,ACT_CODE,ACT_TITLE,ACT_DESCRIPTION,ACT_WEB_ADDRESS
0,117,Research Construction Programs,C06,Research Facilities Construction Grant,"To provide matching Federal funds, up to 75%, ...",NaN
1,1,Institutional Training and Director Program Pr...,D43,International Research Training Grants,To support research training programs for US a...,http://grants.nih.gov/training/F_files_non_nrs...
2,2,Institutional Training and Director Program Pr...,D71,International Research Training Planning Grant,To plan for the preparation of an application ...,http://grants.nih.gov/training/F_files_non_nrs...
3,3,Institutional Training and Director Program Pr...,DP1,NIH Director’s Pioneer Award (NDPA),To support individuals who have the potential ...,http://commonfund.nih.gov/pioneer/
4,4,Institutional Training and Director Program Pr...,DP2,NIH Director’s New Innovator Awards,To support highly innovative research projects...,http://commonfund.nih.gov/newinnovator/


4. API  
https://api.reporter.nih.gov/

will get a holistic dataset on projects through API. Submit query and obtain json。
However, there is an entry limit of 500 imposed on each query. 

In [31]:


headers = {
    'accept': 'application/json',
    # Already added when you pass json=
    # 'Content-Type': 'application/json',
}

json_data = {
    'searchId': '*',
    'criteria': {
        'useRelevance': True,
        'includeActiveProjects': True,
        'offset': 0,
        'limit': 3
    },
}

response = requests.post('https://api.reporter.nih.gov/v2/projects/search', headers=headers, json=json_data)

In [34]:

dict = json.loads(response.text)
df = pd.json_normalize(dict['results'])
df.head()

,appl_id,subproject_id,fiscal_year,project_num,project_serial_num,award_type,activity_code,award_amount,is_active,project_num_split,...,full_study_section.irg_code,full_study_section.srg_code,full_study_section.srg_flex,full_study_section.sra_designator_code,full_study_section.sra_flex_code,full_study_section.group_code,full_study_section.name,full_study_section.url,full_study_section.cmte_id,full_study_section.cluster_irg_code
0,9795459,None,2018,261201500036I-0-26100009-2,None,None,N01,930464.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10324021,None,2021,1R43OH012283-01,OH012283,1,R43,243241.0,False,NaN,...,ZRG1,ZRG1,,AIDC,V,13,Special Emphasis Panel[ZRG1 AIDC-V (13)],NaN,NaN,NaN
2,10294664,None,2020,75N92020P00309-0-0-3,None,None,N02,100000.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10287968,None,2020,75N95019D00013-0-759502000001-1,None,None,N01,2113098.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10329701,None,2021,75N95019D00013-P00002-759502000001-1,None,None,N01,549285.0,False,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
df.columns

NameError: name 'df' is not defined

In [388]:
pi = pd.read_csv('pi.csv')

In [390]:
pi.head()

,profile_id,FY,ETHNICITY2,gender,DEGREE,tot_doll,race,age,number_rpg_desc
0,8dcd5fb732c6c400e5f6f821dd07f494,1985,Withheld,Male,PhD,2.858527e+05,White,Early,One
1,3211e3aac113bb58c7a31d1756aa4d22,1985,Unknown,Female,MD,3.364276e+05,Unknown,Mid,One
2,cf3b26ce35fbd4519df488f89bab3953,1985,Unknown,Male,PhD,6.722707e+05,White,Mid,Two
3,d9a01ca480f0b65f62ede6b3a6b80cd5,1985,Not Hispanic,Male,MD,7.244536e+06,White,Mid,Two
4,dbfeefc788f02e51ab94cce0d056b65c,1985,Not Hispanic,Male,PhD,1.612166e+05,White,Early,One


In [392]:
org = pd.read_csv('anon_org.csv')
org.head()

,IPF,FY,ORG_TYPE,REGION,STATE_CODE,tot_doll
0,8d77e73daef9e18b3c0cf49cfce53deb,1985,Foreign,Foreign,NaN,2.527544e+05
1,329cf1e21fdbb919ff0edbefecf17593,1985,Foreign,Foreign,NaN,5.020836e+04
2,cf4c8f5328bc7625a579a48b21782ad7,1985,Other Domestic NonProfit,North East,PA,2.952087e+05
3,d98e02f89718ab8ea2e487cc6a78d3d3,1985,Other Higher Education,North East,NY,2.386051e+05
4,db02da514842b5fa45c1ae78b00c9eab,1985,Medical Schools,Mid West,WI,2.987367e+07
